# Cascade decay of a Ne 1s-hole: Compute the cascade tree

# **This nootebook is still under construction !!!**

In [2]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13735060134599690084\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6517063341696747294\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-13735060134599690084\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


The neon 1s-cascade has attracted interest in the literature. ... Different observables and spectra have been recorded by experiment.

In JAC, two steps are generally distinguished : (1) computation of the cascade tree; (2) the simulation of spectra and observables by making use of these trees.

In this notebook, we shall first consider on the (1) computation of the cascade tree; ... what does this include from a simple physics perspectives: automatic generation of configurations that can be successively be reached by various processes. For instance, while the radiative transition just leads to some de-excitation of the atomic state, Auger (and other processes) will lead to the emission of electrons and, hence, to final-state configuration with one or a few electrons less.

In JAC, the calculation of a cascade tree is organized as a `Cascade.Computation()`. This is a container that helds all information about the cascade as easily provided from the viewpoint of experiment. ... What does it include: Let us have a look at the definition of such a `Cascade.Computation`:


In [3]:
? Cascade.Computation

`struct  Cascade.Computation`       ... defines a type for a cascade computation, i.e. for the computation of a whole excitation and/or decay cascade. The data          from this computation can be modified, adapted and refined to the practical needs before the actual computations are          carried out. Initially, this struct contains the physical metadata about the cascade to be calculated but gets enlarged          in course of the computation to keep also wave functions, level multiplets, etc.

```
+ name               ::String                         ... A name for the cascade
+ nuclearModel       ::Nuclear.Model                  ... Model, charge and parameters of the nucleus.
+ grid               ::Radial.Grid                    ... The radial grid to be used for the computation.
+ asfSettings        ::AsfSettings                    ... Provides the settings for the SCF process.
+ approach           ::Cascade.AbstractApproach       ... Computational approach/model that is applied to generate and evaluate the 
                                                          cascade; possible approaches are: {'single-configuration', ...}
+ processes          ::Array{Basics.AtomicProcess,1}   ... List of the atomic processes that are supported and should be included into the 
                                                          cascade.
+ initialConfs       ::Array{Configuration,1}         ... List of one or several configurations that contain the level(s) from which the 
                                                          cascade starts.
+ initialLevels      ::Array{Tuple{Int64,Float64},1}  ... List of one or several (tupels of) levels together with their relative population 
                                                          from which the cascade starts.
+ maxElectronLoss    ::Int64                          ... (Maximum) Number of electrons in which the initial- and final-state 
                                                          configurations can differ from each other; this also determines the maximal steps 
                                                          of any particular decay path.
+ NoShakeDisplacements ::Int64                        ... Maximum number of electron displacements due to shake-up or shake-down processes 
                                                          in any individual step of the cascade.
+ shakeFromShells ::Array{Shell,1}                    ... List of shells from which shake transitions may occur.
+ shakeToShells   ::Array{Shell,1}                    ... List of shells into which shake transitions may occur.
+ steps           ::Array{Cascade.Step,1}             ... List of individual steps between well-defined atomic multiplets that are 
                                                          included into the cascade.
```

---

`Cascade.Computation()`  ... constructor for an 'empty' instance of a Cascade.Computation.


What is seen ... and which information needed to be provided for such a computation